In [1]:
import os
import numpy as np 
import pandas as pd

#image reading
import pydicom # for DICOM images

!pip install python-gdcm -q
!pip install pylibjpeg -q 

# my utility files
import preprocessrsna_py as pre
import resnetrsna_py as rsmodel
import trainrsna_py as trn

# for reading yaml and storing configuration 
import argparse
import yaml
import sys

import torch.nn as nn
import torch.nn.functional as F
from torch import FloatTensor, LongTensor
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Data Augmentation for Image Preprocessing
from albumentations import (ToFloat, Normalize, VerticalFlip, HorizontalFlip, Compose, Resize,
                            RandomBrightnessContrast, HueSaturationValue, Blur, GaussNoise,
                            Rotate, RandomResizedCrop, Cutout, ShiftScaleRotate, ToGray)
from albumentations.pytorch import ToTensorV2

#from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet50   # resnet34

# SKlearn
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score,  roc_auc_score, confusion_matrix

In [2]:
inpath = "/kaggle/input/rsna-breast-cancer-detection"
train = pre.preprocess(inpath)

In [ ]:
#model
output_size = 1
no_columns = len(trn.csv_columns) # csv_columns defined in trn

model1 = rsmodel.ResNet50Network(output_size=output_size, no_columns=no_columns)

# data - new_df is a smaller dataframe to check that everything works. 
# Use full dataset for actual training.
new_df = train.groupby('cancer').apply(lambda x: x.sample(n=24)).reset_index(drop = True)

# configuration
sys.argv = ['-f']
config_path = '/kaggle/input/config1/config1.yaml'
args = trn.set_params(config_path)

#train
trn.train_folds(args, model=model1, train_original=new_df)